In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
%load_ext autotime

import seaborn as sns
sns.set(color_codes=True)
import matplotlib.pyplot as plt
import numpy as np
plt.rcParams['savefig.dpi'] = 100

from IPython.display import display

from utils.readers import CsvReader
from clfpipeline import Clfpipeline
from classifiers.mlp import MLP
from classifiers.softmax_example import SoftmaxRegression
from classifiers.misc import merge_dicts

In [2]:
csv_reader = CsvReader(has_header=False, fn='input/trunc_mnist/trunc_mnist20x20_inputs.csv', fn_labels='input/trunc_mnist/trunc_mnist20x20_targets.csv')
pipe = Clfpipeline(csv_reader)
pipe.prepare_data()
n_samples = pipe.X_train.shape[0]

# truncmnist

In [2]:
csv_reader = CsvReader(has_header=False, fn='input/trunc_mnist/trunc_mnist20x20_inputs.csv', fn_labels='input/trunc_mnist/trunc_mnist20x20_targets.csv')
pipe = Clfpipeline(csv_reader)
pipe.prepare_data()

In [4]:
conf_clf = {'dims':(400, 50, 10), 
            'alpha':0.001, 
            'reg':0.001}
conf_train = {'batchsize':9,
              'costevery': 10,
               'nepoch': 200}
fullconf = merge_dicts(conf_clf, conf_train)
pipe.set_classifier(SoftmaxRegression(**conf_clf))
pipe.train(**conf_train)
pipe.plot()


In [121]:
pipe.save_weights(algo_name="LR", dataset_name="TRUNCMNIST", confs=fullconf, folder="output/truncmnist/")

# Iris Dataset

In [97]:
csv_reader = CsvReader(has_header=False, fn='input/iris/iris-data-num_lab.csv', label_pos=-1)
pipe = Clfpipeline(csv_reader)
pipe.prepare_data()

In [98]:
conf_clf = {'dims':(4, 7, 3), 
            'alpha':0.01, 
            'reg':0.01}

conf_train = {'batchsize':-1,
              'costevery': 50,
               'nepoch': 300}
fullconf = merge_dicts(conf_clf, conf_train)

pipe.set_classifier(MLP(**conf_clf))
pipe.train(**conf_train)
pipe.plot()

In [95]:
pipe.save_weights(algo_name="MLP", dataset_name="IRIS", confs=fullconf, folder="output/truncmnist/")

# Full mnist

In [135]:
csv_reader = CsvReader(has_header=False, fn='input/full_mnist/fullmnist_inp_transp', fn_labels='input/full_mnist/fullMNIST_labels.csv')
pipe = Clfpipeline(csv_reader)
pipe.prepare_data()

In [ ]:
conf_clf = {'dims':(784, 20, 10), 
            'alpha':0.05, 
            'reg':0.0001}

conf_train = {'batchsize':10,
              'costevery': 1,
               'nepoch': 300}
fullconf = merge_dicts(conf_clf, conf_train)

pipe.set_classifier(MLP(**conf_clf))
pipe.train(**conf_train)
pipe.plot()

In [125]:


import pandas as pd
fullmnist = pd.read_csv('input/full_mnist/fullMNIST_inputs.csv', header=None)

In [134]:
np.savetxt("fullmnist_inp_transp", fullmnist.as_matrix().T, delimiter= ',', fmt='%d')

## Autoencoder_Sparse


In [4]:
from classifiers.autoencoder_sparse import AutoEncoderSparse
from utils.preprocessors import AutoEncoderPP

csv_reader = CsvReader(has_header=False, fn='input/trunc_mnist/trunc_mnist20x20_inputs.csv')
pipe = Clfpipeline(csv_reader, PreProc=AutoEncoderPP)
pipe.prepare_data()

In [55]:
ac = AutoEncoderSparse(**conf_clf)
ac.grad_check(X, X)

In [10]:
conf_clf = {'dims':(400, 32, 400), 
            'alpha':0.00005, 
            'reg':0.0000,
            'beta':0,
            'ro':0.05}

conf_train = {'batchsize':-1,
              'costevery': 1,
               'nepoch': 300,
                'acc_batch':True}
fullconf = merge_dicts(conf_clf, conf_train)

#aec = AutoEncoderSparse(**conf_clf)
#aec.grad_check(pipe.X[0], pipe.X[0])
pipe.set_classifier(AutoEncoderSparse(**conf_clf))
pipe.train(**conf_train)
#pipe.plot()

### Try with Keras

In [71]:
pipe.save_weights(algo_name="AE_SPARSE", dataset_name="TRUNCMNIST", confs=fullconf, folder="output/truncmnist/")

In [11]:
pipe.plot()

In [12]:
X = pipe.preprocessor.X[4]

In [26]:
np.min(pipe.preprocessor.X)
#np.max(pipe.preprocessor.X)

In [53]:
mn = np.min(pipe.preprocessor.X)
mx = np.max(pipe.preprocessor.X)
print mn, mx
def num_to_col(num):
    return 255. / (mx - mn) * (num + np.abs(mn))
    

In [43]:
vfunc = np.vectorize(num_to_col)
yy = vfunc(X)

In [49]:
pipe.preprocessor.X[2].shape

In [59]:
def vis_images(dataset, img_n, img_m, n):
    """ Visualize input and output images of autoencoder
    img_n, img_m : image dimensions
    n : number of images to display
    """

    plt.figure(figsize=(20, 4))
    for i in range(n):
        # display original
        ax = plt.subplot(2, n, i + 1)
        mg = vfunc(dataset[i])
        plt.imshow(mg.reshape(img_n, img_m))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        # display reconstruction
        ax = plt.subplot(2, n, i + 1 + n)
        dmg, _ = pipe.classifier.predict_single(dataset[i])
        plt.imshow(dmg.reshape(img_n, img_m))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()
    
vis_images(pipe.preprocessor.X, 20, 20, 5)




In [15]:
plt.bar(range(len(X)), np.abs(X - XX))
#sns.heatmap()
plt.show()
#plt.bar(range(len(X)), np.abs(X))
print np.sum(X**2)
print np.sum((X - XX) ** 2)